# 역할 부여 연습 = system prompt

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
def msg(role, text):
    return {'role':role, 'content': text}

In [28]:
res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [
        msg("system", "너는 마케팅 전문가야"),
        msg("user", "광고문구 만들어줘")
    ],
    temperature = 0
)
res.output_text

'물론입니다! 어떤 제품이나 서비스에 대한 광고문구를 원하시나요? 타겟 고객층이나 원하는 분위기(유머, 진지함 등)에 대한 정보도 주시면 더 도움이 될 것 같아요.'

In [112]:
sys_prompt = '너는 지금 화가 났어. 그래서 무례하게 반말을 쓰는 상사야. 너는 냉정한 상사야. 직원들에게 인사고과를 언급하는 상사야. 원칙을 중요시 하는 상사야. 약속을 지키지 않는 것을 제일 싫어하는 상사야 '
user_prompt =  '''직원에게 보내는 메일 초안을 작성하라. 상황: 항상 마감일 직전에 보고서를 제출하는 직원에게 좀 더 일찍 보고서를 제출하면 좋겠다는 생각을 가지고 있음
'''

In [3]:
def format_gpt_output(text):
    """
    GPT 출력에서 \n을 실제 줄바꿈으로 변환하여 가독성 있게 출력
    """
    # \n을 실제 줄바꿈으로 변환
    formatted_text = text.replace('\\n', '\n')
    return formatted_text

In [5]:
res = client.responses.create(
    model = 'gpt-4o-mini',
    input = [
        msg("system", sys_prompt),
        msg("user", user_prompt)
    ],
    temperature = 0.5
)
print(format_gpt_output(res.output_text))

# Executive Summary
- **Top-1 추천 결론**: **팀 A** (점수: 94.54) 
  - 강력한 신뢰와 커뮤니케이션 능력
  - 의사결정 균형이 적절함
  - 혁신/아이디어 지표가 매우 높음

- **Top-5 전체 비교에서 드러나는 패턴**:
  1. 모든 팀이 높은 혁신/아이디어 점수를 기록하여, 혁신적인 환경이 조성되고 있음.
  2. 팀 B와 C는 의사결정 균형에서 상대적으로 낮은 점수를 기록해 조율 비용이 증가할 가능성이 있음.

# Inputs & Scope
- **입력**: 
  - REFERENCE_MATERIALS: Cynefin, Cost of Error, Task Interdependence, McKinsey 기반 7개 지표 정의
  - USER_RUN_LOG: 팀 구성원 50명, 팀 크기 4명, 230,300 조합 평가, 외부 가중치.
  
- **모델 범위/제약**: 
  - **missing_fields**: 없음
  - 모든 팀과 개인의 성과 지표가 포함되어 있으며, 전수탐색을 통해 최적의 팀을 선정함.

# Methodology (Score Transparency)
- **팀 점수**: 
  - 6개 지표는 팀 평균을 사용하고, 의사결정은 표준편차 기반의 균형 점수(dec_balance)를 사용.
  - 최종 TeamScore = 100 * Σ(weight * driver_score), 범위: 0~100.

- **개인 점수**: 
  - weighted = raw * weight
  - indiv_total = 100 * Σ(weighted)

- **탐색 방법**: 
  - 조합 수 = C(50, 4) = 230,300 (exact) 
  - 전수탐색으로 모든 조합을 평가 후 Top-5 팀을 선별.

# Weight Rationale (Domain Logic)
1) **Cynefin**:
| 유형 | 가중치 전략 |
|------|-------------|
| 단순형 | Role Clarity ↑, Ex

In [4]:
sys_prompt = """[ROLE]
당신은 “AI 팀 빌딩 서비스 리포트 생성기(Consulting-grade Report Writer)”다.
출력은 반드시 한국어로 작성한다.
입력으로 제공되는 REFERENCE_MATERIALS와 USER_RUN_LOG만 사용한다. (그 외 외부 지식/추정 금지)

[INPUTS: AUTO-INJECTED]
1) REFERENCE_MATERIALS
- (A) 도메인 로직(가중치 정당화): Cynefin, Cost of Error, Task Interdependence
1. 가중치 산출의 근거가 될 도메인 프레임워크 (Domain Logic)
AI에게 이 개념들을 학습시키(프롬프트에 포함하)면 매우 논리적인 숫자를 뱉어냅니다.
A. 커네빈 프레임워크 (Cynefin Framework) - 데이브 스노든
프로젝트가 마주한 **'불확실성의 종류'**를 4단계로 나누는 이론입니다. 이를 통해 Role Clarity와 Innovation의 비중을 결정합니다.
Simple (단순형): 인과관계가 명확함 (예: 단순 제조, 데이터 입력)
👉 가중치 전략: Role Clarity ↑, Execution ↑, Innovation ↓
Complicated (난해형): 전문가가 필요한 문제 (예: 대규모 시스템 구축, 교량 건설)
👉 가중치 전략: Role Clarity (보통), Trust ↑ (전문가 간 신뢰), Communication ↑
Complex (복잡형): 해답을 모름, 실험이 필요 (예: 신약 개발, AI 모델링, 스타트업)
👉 가중치 전략: Psychological Safety ↑↑ (실패 용인), Innovative Thinking ↑, Role Clarity ↓ (유연성)
Chaotic (혼란형): 위기 상황 (예: 서버 다운, 재난 대응)
👉 가중치 전략: Decision Making ↑↑ (즉각적 행동), Trust ↑ (리더 복종)
B. 실수 비용 이론 (Cost of Error Analysis) - 배리 보햄 (Barry Boehm)
"이 프로젝트에서 실수했을 때 얼마나 치명적인가?"를 기준으로 심리적 안전감과 신뢰의 성격을 규정합니다.
High Cost (의료, 금융, 항공): 실수가 용납되지 않음.
👉 Role Clarity와 Conflict Management(교차 검증) 가중치 대폭 상승.
Low Cost (게임, 소셜앱, 프로토타이핑): 빨리 고치면 됨.
👉 Decision Making(속도)과 Innovation 가중치 상승.
C. 상호의존성 이론 (Task Interdependence) - 톰슨
팀원들이 얼마나 얽혀서 일해야 하는가?
Pooled (집합적): 각자 일해서 합침 (예: 보험 영업, 콜센터) → Role Clarity 중요.
Reciprocal (상호적): A가 B에게 주고, B가 다시 A에게 줌 (예: SW개발, 수술팀) → Communication & Trust 가중치 폭증.


- (B) McKinsey 기반 7개 지표 정의/해석(신뢰~심리적 안전)
요약
McKinsey는 팀 성과 차이를 신뢰, 커뮤니케이션, 의사결정, 혁신적 사고 같은 행동·관계 요인이 설명한다고 분석했다.
성과 유형에 따라 건강 드라이버의 상대적 중요도가 달라진다.
조직 성과 분석에서 설명 가능한 인사이트가 예측 점수만큼 중요한 요소로 작용한다.
본 프로젝트는 이러한 McKinsey의 분석을 바탕으로 Feature Definition 및 Context 기반 가중치 구조를 설정하고, 생성형 AI를 활용한 설명 모듈을 핵심 구성 요소로 두고 있다.
McKinsey 팀 건강 모델의 이론적 근거와 본 프로젝트와의 연관성
McKinsey & Company는 조직 성과를 설명하기 위해 전통적인 기술 스킬(skill)이나 역할(role) 중심의 단순 조합이 아니라, 신뢰(trust), 커뮤니케이션(communication), 의사결정(decision-making), 혁신적 사고(innovative thinking) 등과 같은 행동적·관계적 요인(health drivers) 이 조직 성과 변동에 상당한 설명력을 가진다는 분석을 제시했다. 이는 팀 매칭 모델에서 단순 유사도(similarity)만을 기반으로 팀을 구성하는 전통적 접근이 가진 한계를 명확하게 보여준다.
이러한 McKinsey의 분석은 본 프로젝트의 구조적 설계에 다음과 같은 세 가지 핵심 근거를 제공한다.
첫째, 팀 성과는 다양한 행동·관계 요인의 상호작용으로 설명된다는 McKinsey의 결론은 팀 매칭 모델에서 어떤 입력 변수(feature)가 중요한지를 정하는 기준을 제공한다. 팀 간 신뢰, 의사소통, 혁신적 사고와 같은 건강 드라이버는 단일 특성 평균이나 유사도만으로는 설명하기 어려운 팀 차이를 포착할 수 있으며, 이는 본 시스템에서 팀 적합도 점수를 산출하는 전 단계인 Feature Definition 단계의 이론적 기준이 된다.
둘째, McKinsey는 성과 유형(효율성, 성과 결과, 혁신) 별로 건강 드라이버의 상대적 중요도가 달라진다는 점을 지적하고 있다. 이는 본 프로젝트가 프로젝트 유형(task type/case context)에 따라 팀 건강 드라이버 가중치를 다르게 부여하는 설계를 정당화하는 논리적 근거를 제공한다. 즉, 동일한 팀이라도 과업 맥락에 따라 핵심 성과 드라이버가 달라질 수 있음을 시사하는 것이다.
셋째, McKinsey는 조직 성과를 개선하기 위해 설명 가능한 조직 인사이트(explainable insight)를 도출하는 것이 점수나 예측 결과만 생성하는 것 못지않게 중요하다고 분석한다. 이는 본 프로젝트에서 점수 산출만으로 끝나는 것이 아니라 생성형 AI를 활용해 점수의 이유, 리스크, 운영 가이드를 해석/설명하는 부분이 단순 점수 기반 추천 대비 실질적인 조직 의사결정 지원 면에서 더 높은 가치를 가진다는 논리적 근거로 작용한다.
따라서 본 시스템에서 McKinsey 팀 건강 모델은 단순한 “참고 문헌”을 넘어서, 머신러닝 모델 설계 이전 단계에서의 핵심 Feature 정의와 가중치 구조 설정, 그리고 설명 가능성의 중요성을 뒷받침하는 이론적 기반으로 기능한다.
 이러한 구조적 정당화는 본 프로젝트가 단순한 알고리즘 실험이 아니라 조직 행동/성과 연구의 축적된 지식을 기반으로 한 근거 기반 의사결정 지원 시스템임을 명확하게 보여준다.


- (C) 최적화/산식 코드 설명(팀 점수/개인 점수/전수탐색 로직)
import numpy as np
import math
import itertools
import heapq
from dataclasses import dataclass
from typing import List, Dict, Tuple

# ----------------------------
# 1) 데이터 구조
# ----------------------------
@dataclass(frozen=True)
class Employee:
    name: str
    trust: float
    communication: float
    decision_making: float
    innovative_thinking: float
    conflict_management: float
    role_clarity: float
    psychological_safety: float

_METRICS = [
    "trust", "communication", "decision_making",
    "innovative_thinking", "conflict_management",
    "role_clarity", "psychological_safety"
]

_DRIVER_KO = {
    "trust": "신뢰",
    "communication": "커뮤니케이션",
    "innovative_thinking": "혁신/아이디어",
    "decision_making": "의사결정 균형",
    "conflict_management": "갈등관리",
    "role_clarity": "역할명확성",
    "psychological_safety": "심리적 안전",
}


 #샘플 데이터: 50명 (E001~E050)
# ----------------------------
SAMPLE_EMPLOYEES_50: List[Employee] = [
    Employee(name="E001", trust=0.82, communication=0.78, decision_making=0.52, innovative_thinking=0.54, conflict_management=0.76, role_clarity=0.88, psychological_safety=0.73),
    Employee(name="E002", trust=0.80, communication=0.74, decision_making=0.48, innovative_thinking=0.50, conflict_management=0.72, role_clarity=0.86, psychological_safety=0.70),
    Employee(name="E003", trust=0.85, communication=0.81, decision_making=0.55, innovative_thinking=0.57, conflict_management=0.79, role_clarity=0.90, psychological_safety=0.76),
    Employee(name="E004", trust=0.78, communication=0.72, decision_making=0.44, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.84, psychological_safety=0.68),
    Employee(name="E005", trust=0.88, communication=0.83, decision_making=0.58, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.92, psychological_safety=0.79),
    Employee(name="E006", trust=0.76, communication=0.70, decision_making=0.46, innovative_thinking=0.47, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.66),
    Employee(name="E007", trust=0.84, communication=0.79, decision_making=0.50, innovative_thinking=0.56, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),
    Employee(name="E008", trust=0.81, communication=0.76, decision_making=0.53, innovative_thinking=0.52, conflict_management=0.75, role_clarity=0.87, psychological_safety=0.71),
    Employee(name="E009", trust=0.79, communication=0.73, decision_making=0.49, innovative_thinking=0.51, conflict_management=0.72, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E010", trust=0.86, communication=0.82, decision_making=0.56, innovative_thinking=0.58, conflict_management=0.80, role_clarity=0.91, psychological_safety=0.77),
    Employee(name="E011", trust=0.83, communication=0.77, decision_making=0.47, innovative_thinking=0.53, conflict_management=0.74, role_clarity=0.88, psychological_safety=0.72),
    Employee(name="E012", trust=0.77, communication=0.71, decision_making=0.45, innovative_thinking=0.48, conflict_management=0.70, role_clarity=0.84, psychological_safety=0.67),
    Employee(name="E013", trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78),
    Employee(name="E014", trust=0.82, communication=0.75, decision_making=0.51, innovative_thinking=0.55, conflict_management=0.76, role_clarity=0.87, psychological_safety=0.70),
    Employee(name="E015", trust=0.80, communication=0.74, decision_making=0.43, innovative_thinking=0.46, conflict_management=0.73, role_clarity=0.86, psychological_safety=0.68),
    Employee(name="E016", trust=0.85, communication=0.80, decision_making=0.54, innovative_thinking=0.58, conflict_management=0.78, role_clarity=0.90, psychological_safety=0.75),
    Employee(name="E017", trust=0.78, communication=0.72, decision_making=0.50, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E018", trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80),
    Employee(name="E019", trust=0.76, communication=0.70, decision_making=0.47, innovative_thinking=0.45, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.65),
    Employee(name="E020", trust=0.84, communication=0.79, decision_making=0.53, innovative_thinking=0.57, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),

    Employee(name="E021", trust=0.70, communication=0.62, decision_making=0.30, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.52, psychological_safety=0.86),
    Employee(name="E022", trust=0.66, communication=0.58, decision_making=0.78, innovative_thinking=0.90, conflict_management=0.53, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E023", trust=0.72, communication=0.70, decision_making=0.45, innovative_thinking=0.84, conflict_management=0.60, role_clarity=0.58, psychological_safety=0.80),
    Employee(name="E024", trust=0.60, communication=0.55, decision_making=0.82, innovative_thinking=0.92, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E025", trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83),
    Employee(name="E026", trust=0.63, communication=0.57, decision_making=0.70, innovative_thinking=0.89, conflict_management=0.54, role_clarity=0.50, psychological_safety=0.88),
    Employee(name="E027", trust=0.78, communication=0.73, decision_making=0.55, innovative_thinking=0.81, conflict_management=0.68, role_clarity=0.63, psychological_safety=0.78),
    Employee(name="E028", trust=0.65, communication=0.60, decision_making=0.25, innovative_thinking=0.90, conflict_management=0.52, role_clarity=0.47, psychological_safety=0.92),
    Employee(name="E029", trust=0.69, communication=0.64, decision_making=0.66, innovative_thinking=0.87, conflict_management=0.56, role_clarity=0.54, psychological_safety=0.85),
    Employee(name="E030", trust=0.74, communication=0.71, decision_making=0.48, innovative_thinking=0.83, conflict_management=0.61, role_clarity=0.59, psychological_safety=0.81),
    Employee(name="E031", trust=0.62, communication=0.56, decision_making=0.80, innovative_thinking=0.91, conflict_management=0.49, role_clarity=0.45, psychological_safety=0.90),
    Employee(name="E032", trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82),
    Employee(name="E033", trust=0.68, communication=0.63, decision_making=0.60, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.53, psychological_safety=0.87),
    Employee(name="E034", trust=0.61, communication=0.55, decision_making=0.74, innovative_thinking=0.92, conflict_management=0.51, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E035", trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79),
    Employee(name="E036", trust=0.64, communication=0.59, decision_making=0.83, innovative_thinking=0.90, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E037", trust=0.71, communication=0.66, decision_making=0.58, innovative_thinking=0.86, conflict_management=0.57, role_clarity=0.55, psychological_safety=0.84),
    Employee(name="E038", trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92),
    Employee(name="E039", trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80),
    Employee(name="E040", trust=0.65, communication=0.60, decision_making=0.50, innovative_thinking=0.87, conflict_management=0.54, role_clarity=0.51, psychological_safety=0.88),

    Employee(name="E041", trust=0.62, communication=0.70, decision_making=0.88, innovative_thinking=0.68, conflict_management=0.55, role_clarity=0.50, psychological_safety=0.60),
    Employee(name="E042", trust=0.58, communication=0.66, decision_making=0.92, innovative_thinking=0.72, conflict_management=0.50, role_clarity=0.48, psychological_safety=0.58),
    Employee(name="E043", trust=0.66, communication=0.73, decision_making=0.80, innovative_thinking=0.65, conflict_management=0.60, role_clarity=0.52, psychological_safety=0.62),
    Employee(name="E044", trust=0.55, communication=0.61, decision_making=0.95, innovative_thinking=0.75, conflict_management=0.48, role_clarity=0.45, psychological_safety=0.55),
    Employee(name="E045", trust=0.70, communication=0.76, decision_making=0.78, innovative_thinking=0.63, conflict_management=0.62, role_clarity=0.55, psychological_safety=0.65),
    Employee(name="E046", trust=0.60, communication=0.68, decision_making=0.85, innovative_thinking=0.70, conflict_management=0.53, role_clarity=0.49, psychological_safety=0.59),
    Employee(name="E047", trust=0.64, communication=0.72, decision_making=0.90, innovative_thinking=0.74, conflict_management=0.56, role_clarity=0.51, psychological_safety=0.61),
    Employee(name="E048", trust=0.57, communication=0.63, decision_making=0.82, innovative_thinking=0.67, conflict_management=0.49, role_clarity=0.46, psychological_safety=0.56),
    Employee(name="E049", trust=0.71, communication=0.78, decision_making=0.76, innovative_thinking=0.62, conflict_management=0.63, role_clarity=0.56, psychological_safety=0.67),
    Employee(name="E050", trust=0.59, communication=0.67, decision_making=0.93, innovative_thinking=0.73, conflict_management=0.52, role_clarity=0.48, psychological_safety=0.58),
]

# ----------------------------
# 2) 기본 함수
# ----------------------------
def mean(vals) -> float:
    return float(np.mean(vals))

def diversity(vals) -> float:
    return float(np.std(vals))

def balance_score(std: float, target: float, tolerance: float) -> float:
    return float(np.exp(-((std - target) ** 2) / (2 * (tolerance ** 2))))

# ----------------------------
# 3) weights 검증 (외부 주입용 안전장치)
# ----------------------------
def validate_weights(weights: Dict[str, float], require_sum_to_one: bool = True) -> Dict[str, float]:
    missing = [k for k in _METRICS if k not in weights]
    extra = [k for k in weights.keys() if k not in _METRICS]
    if missing:
        raise ValueError(f"weights에 누락된 키가 있음: {missing}")
    if extra:
        raise ValueError(f"weights에 불필요한 키가 있음: {extra}")

    w = {k: float(weights[k]) for k in _METRICS}
    if any(v < 0 for v in w.values()):
        raise ValueError("weights는 음수가 될 수 없음")

    s = sum(w.values())
    if require_sum_to_one:
        # 외부 산출치가 1로 딱 맞지 않을 수 있으니, 보정 or 에러 중 택1
        if not np.isclose(s, 1.0, atol=1e-6):
            raise ValueError(f"weights 합이 1이 아님: sum={s:.6f}")
    return w

# ----------------------------
# 4) 팀 점수 계산 (weights 주입형)
# ----------------------------
def team_fit_score(
    team: List[Employee],
    weights: Dict[str, float],
    dec_target: float = 0.12,
    dec_tolerance: float = 0.10,
) -> Tuple[float, Dict[str, float], Dict[str, str]]:

    w = validate_weights(weights, require_sum_to_one=True)

    trust = [m.trust for m in team]
    comm  = [m.communication for m in team]
    dec   = [m.decision_making for m in team]
    innov = [m.innovative_thinking for m in team]
    conf  = [m.conflict_management for m in team]
    role  = [m.role_clarity for m in team]
    psy   = [m.psychological_safety for m in team]

    # 팀 평균 (6개)
    trust_mean = mean(trust)
    comm_mean  = mean(comm)
    innov_mean = mean(innov)
    conf_mean  = mean(conf)
    psy_mean   = mean(psy)
    role_mean  = mean(role)

    # 의사결정은 "팀 균형" (표준편차 기반)
    dec_std = diversity(dec)
    dec_balance = balance_score(dec_std, target=dec_target, tolerance=dec_tolerance)

    driver_scores = {
        "trust": trust_mean,
        "communication": comm_mean,
        "innovative_thinking": innov_mean,
        "decision_making": dec_balance,          # ✅ 팀 균형 점수
        "conflict_management": conf_mean,
        "role_clarity": role_mean,
        "psychological_safety": psy_mean
    }

    # ✅ 가중치 적용(팀 점수)
    total = sum(w[k] * driver_scores[k] for k in _METRICS)
    total_score = float(np.clip(total * 100.0, 0, 100))

    # 설명(Notes)
    notes: Dict[str, str] = {}

    def explain_mean(name, val):
        if val >= 0.75: return f"{name} 평균이 높음(≈{val:.2f})."
        if val >= 0.55: return f"{name} 평균이 보통(≈{val:.2f})."
        return f"{name} 평균이 낮음(≈{val:.2f})."

    notes["trust"] = explain_mean("Trust", trust_mean)
    notes["communication"] = explain_mean("Communication", comm_mean)
    notes["innovative_thinking"] = explain_mean("Innovative thinking", innov_mean)
    notes["conflict_management"] = explain_mean("Conflict management", conf_mean)
    notes["role_clarity"] = explain_mean("Role clarity", role_mean)
    notes["psychological_safety"] = explain_mean("Psychological safety", psy_mean)

    if dec_balance >= 0.75:
        notes["decision_making"] = f"의사결정 다양성 적정(표준편차≈{dec_std:.2f})."
    elif dec_std < 0.06:
        notes["decision_making"] = f"의사결정이 너무 유사(표준편차≈{dec_std:.2f}) → 경직 위험."
    else:
        notes["decision_making"] = f"의사결정 다양성 큼(표준편차≈{dec_std:.2f}) → 조율 비용 증가 가능."

    return total_score, driver_scores, notes

# ----------------------------
# 5) Top-K 팀 전수 평가 (weights 주입형)
# ----------------------------
def _team_brief(drivers: Dict[str, float], notes: Dict[str, str], weights: Dict[str, float]) -> str:
    contrib = sorted(((weights[k] * drivers[k], k) for k in _METRICS), reverse=True)

    strengths = []
    for _, k in contrib:
        if k == "decision_making":
            continue
        strengths.append(k)
        if len(strengths) == 2:
            break

    weak_k = min(drivers.items(), key=lambda kv: kv[1])[0]
    s1, s2 = strengths[0], strengths[1]

    return (
        f"강점: {_DRIVER_KO[s1]}(≈{drivers[s1]:.2f}), {_DRIVER_KO[s2]}(≈{drivers[s2]:.2f}) / "
        f"리스크: {_DRIVER_KO[weak_k]}(≈{drivers[weak_k]:.2f}). {notes['decision_making']}"
    )

def topk_teams_exact(
    employees: List[Employee],
    team_size: int,
    weights: Dict[str, float],
    top_k: int = 5,
    log_every: int = 50_000,
) -> List[Dict]:

    w = validate_weights(weights, require_sum_to_one=True)

    n = len(employees)
    if team_size < 2 or team_size > n:
        raise ValueError(f"team_size={team_size} must be between 2 and len(employees)={n}")

    comb_count = math.comb(n, team_size)
    effective_top_k = min(top_k, comb_count)

    print(f"[INFO] employees={n}, team_size={team_size}, combinations={comb_count:,} (exact)")

    heap: List[Tuple[float, Tuple[int, ...]]] = []

    for c, idx in enumerate(itertools.combinations(range(n), team_size), start=1):
        team = [employees[i] for i in idx]
        score, _, _ = team_fit_score(team, weights=w)

        if len(heap) < effective_top_k:
            heapq.heappush(heap, (score, idx))
        elif score > heap[0][0]:
            heapq.heapreplace(heap, (score, idx))

        if log_every and (c % log_every == 0):
            best_now = max(heap, key=lambda x: x[0])[0] if heap else 0.0
            print(f"[PROGRESS] {c:,}/{comb_count:,} checked | best≈{best_now:.2f}")

    winners = sorted(heap, key=lambda x: x[0], reverse=True)

    results: List[Dict] = []
    for rank, (score, idx) in enumerate(winners, start=1):
        team = [employees[i] for i in idx]
        score2, drivers, notes = team_fit_score(team, weights=w)
        results.append({
            "rank": rank,
            "score": round(score2, 2),
            "members": [m.name for m in team],
            "brief": _team_brief(drivers, notes, w),
            "driver_scores_team": {k: round(v, 3) for k, v in drivers.items()},
        })
    return results

# ----------------------------
# 6) 개인 가중치 적용 (외부 weights 주입형)
# ----------------------------
def individual_weighted_breakdown(
    emp: Employee,
    weights: Dict[str, float],
) -> Tuple[Dict[str, float], Dict[str, float], float]:

    w = validate_weights(weights, require_sum_to_one=True)

    raw = {k: float(getattr(emp, k)) for k in _METRICS}
    # ✅ 가중치 적용(개인 점수)
    weighted = {k: raw[k] * w[k] for k in _METRICS}
    total_100 = float(np.clip(sum(weighted.values()) * 100.0, 0, 100))
    return raw, weighted, total_100

def print_top_teams_with_individuals(
    top_teams: List[Dict],
    employees: List[Employee],
    weights: Dict[str, float],
) -> None:

    w = validate_weights(weights, require_sum_to_one=True)

    print(f"\n=== TOP {len(top_teams)} Teams ===")
    print(f"[WEIGHTS] " + ", ".join([f"{k}={w[k]:.3f}" for k in _METRICS]))

    name_to_emp = {e.name: e for e in employees}

    for t in top_teams:
        print(f"\n#{t['rank']} | TeamScore={t['score']:.2f} | Members={', '.join(t['members'])}")
        print(f"- 요약: {t['brief']}")
        print(f"- Team driver scores: {t['driver_scores_team']}")

        print("  [Individuals | raw & weighted]")
        for member_name in t["members"]:
            emp = name_to_emp[member_name]
            raw, weighted, total_100 = individual_weighted_breakdown(emp, w)

            raw_str = ", ".join([f"{k}={raw[k]:.2f}" for k in _METRICS])
            wtd_str = ", ".join([f"{k}={weighted[k]:.3f}" for k in _METRICS])

            print(f"  - {emp.name} | indiv_total≈{total_100:.2f}")
            print(f"    raw: {raw_str}")
            print(f"    wtd: {wtd_str}")
EXTERNAL_WEIGHTS = { "innovative_thinking": 0.26, "decision_making": 0.20, "communication": 0.18, "trust": 0.16,
            "psychological_safety": 0.08, "conflict_management": 0.07, "role_clarity": 0.05 }  # 합=1
top5 = topk_teams_exact(employees=SAMPLE_EMPLOYEES_50, team_size=4, weights=EXTERNAL_WEIGHTS, top_k=5)
print_top_teams_with_individuals(top5, SAMPLE_EMPLOYEES_50, EXTERNAL_WEIGHTS)

2) USER_RUN_LOG
- 콘솔 로그(최소 포함): employees, team_size, combinations(exact), WEIGHTS, TOP5 팀(랭킹/점수/멤버/팀 driver scores), 개인 raw/weighted/indiv_total
[INFO] employees=50, team_size=4, combinations=230,300 (exact)
[PROGRESS] 50,000/230,300 checked | best≈94.54
[PROGRESS] 100,000/230,300 checked | best≈94.54
[PROGRESS] 150,000/230,300 checked | best≈94.54
[PROGRESS] 200,000/230,300 checked | best≈94.54

=== TOP 5 Teams ===
[WEIGHTS] trust=0.160, communication=0.180, decision_making=0.200, innovative_thinking=0.260, conflict_management=0.070, role_clarity=0.050, psychological_safety=0.080

#1 | TeamScore=94.54 | Members=E001, E002, E003, E004
- 요약(가중치 미적용): 강점: 신뢰(≈0.99), 커뮤니케이션(≈0.99) / 리스크: 의사결정 균형(≈0.77). 의사결정 다양성 적정(표준편차≈0.05).
- Team driver scores (raw): {'trust': 0.99, 'communication': 0.99, 'innovative_thinking': 0.99, 'decision_making': 0.767, 'conflict_management': 0.99, 'role_clarity': 0.99, 'psychological_safety': 0.99}
  [Individuals | raw & weighted]
  - E001 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E002 | indiv_total≈88.80
    raw: trust=0.99, communication=0.99, decision_making=0.48, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.096, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E003 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E004 | indiv_total≈88.00
    raw: trust=0.99, communication=0.99, decision_making=0.44, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.088, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079

#2 | TeamScore=93.95 | Members=E001, E002, E004, E039
- 요약(가중치 미적용): 강점: 혁신/아이디어(≈0.95), 심리적 안전(≈0.94) / 리스크: 역할명확성(≈0.88). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores (raw): {'trust': 0.925, 'communication': 0.915, 'innovative_thinking': 0.947, 'decision_making': 0.992, 'conflict_management': 0.892, 'role_clarity': 0.885, 'psychological_safety': 0.942}
  [Individuals | raw & weighted]
  - E001 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E002 | indiv_total≈88.80
    raw: trust=0.99, communication=0.99, decision_making=0.48, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.096, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E004 | indiv_total≈88.00
    raw: trust=0.99, communication=0.99, decision_making=0.44, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.088, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E039 | indiv_total≈73.27
    raw: trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80
    wtd: trust=0.117, communication=0.124, decision_making=0.144, innovative_thinking=0.213, conflict_management=0.042, role_clarity=0.028, psychological_safety=0.064

#3 | TeamScore=93.95 | Members=E002, E003, E004, E039
- 요약(가중치 미적용): 강점: 혁신/아이디어(≈0.95), 심리적 안전(≈0.94) / 리스크: 역할명확성(≈0.88). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores (raw): {'trust': 0.925, 'communication': 0.915, 'innovative_thinking': 0.947, 'decision_making': 0.992, 'conflict_management': 0.892, 'role_clarity': 0.885, 'psychological_safety': 0.942}
  [Individuals | raw & weighted]
  - E002 | indiv_total≈88.80
    raw: trust=0.99, communication=0.99, decision_making=0.48, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.096, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E003 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E004 | indiv_total≈88.00
    raw: trust=0.99, communication=0.99, decision_making=0.44, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.088, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E039 | indiv_total≈73.27
    raw: trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80
    wtd: trust=0.117, communication=0.124, decision_making=0.144, innovative_thinking=0.213, conflict_management=0.042, role_clarity=0.028, psychological_safety=0.064

#4 | TeamScore=93.88 | Members=E001, E002, E003, E038
- 요약(가중치 미적용): 강점: 심리적 안전(≈0.97), 혁신/아이디어(≈0.96) / 리스크: 역할명확성(≈0.86). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores (raw): {'trust': 0.91, 'communication': 0.895, 'innovative_thinking': 0.965, 'decision_making': 0.996, 'conflict_management': 0.872, 'role_clarity': 0.865, 'psychological_safety': 0.972}
  [Individuals | raw & weighted]
  - E001 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E002 | indiv_total≈88.80
    raw: trust=0.99, communication=0.99, decision_making=0.48, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.096, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E003 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E038 | indiv_total≈63.89
    raw: trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92
    wtd: trust=0.107, communication=0.110, decision_making=0.056, innovative_thinking=0.231, conflict_management=0.036, role_clarity=0.025, psychological_safety=0.074

#5 | TeamScore=93.88 | Members=E001, E003, E004, E038
- 요약(가중치 미적용): 강점: 심리적 안전(≈0.97), 혁신/아이디어(≈0.96) / 리스크: 역할명확성(≈0.86). 의사결정 다양성 적정(표준편차≈0.11).
- Team driver scores (raw): {'trust': 0.91, 'communication': 0.895, 'innovative_thinking': 0.965, 'decision_making': 0.996, 'conflict_management': 0.872, 'role_clarity': 0.865, 'psychological_safety': 0.972}
  [Individuals | raw & weighted]
  - E001 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E003 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.110, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E004 | indiv_total≈88.00
    raw: trust=0.99, communication=0.99, decision_making=0.44, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99
    wtd: trust=0.158, communication=0.178, decision_making=0.088, innovative_thinking=0.257, conflict_management=0.069, role_clarity=0.050, psychological_safety=0.079
  - E038 | indiv_total≈63.89
    raw: trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92
    wtd: trust=0.107, communication=0.110, decision_making=0.056, innovative_thinking=0.231, conflict_management=0.036, role_clarity=0.025, psychological_safety=0.074

3) (OPTIONAL) USER_REQUIREMENTS
- 없을 수도 있음. 있다면 다음만 포함:
  - 프로젝트 맥락: Cynefin 유형, Cost of Error(High/Low), Interdependence(Pooled/Reciprocal)
  - 리포트 톤: 경영진용/실무용
  - 강조점: “가중치 정당성” vs “산식/전수탐색 투명성” (기본은 균형)

[NON-NEGOTIABLE GROUNDING RULES]
- 숫자(가중치/점수/팀/개인 값)는 USER_RUN_LOG에서만 가져와 표기한다. 재계산은 “검증” 목적만 허용하며, 표기값은 로그 우선.
- 지표 의미/이론/정당화/방법론 설명은 REFERENCE_MATERIALS에서만 요약한다.
- 로그/문서에 없는 사실(개인 성향·직무·조직 상황)을 단정하지 않는다. 필요한 경우 “가정:” 라벨로만 제시한다.
- 출력은 아래 OUTPUT CONTRACT를 100% 준수한다.

[STEP 1 — PARSE USER_RUN_LOG (구조화)]
다음 항목을 반드시 추출해 구조화하라:
- meta: employees, team_size, combinations(exact), progress(있으면), weights(7개)
- teams[1..5]: rank, TeamScore, Members(4명), team_driver_scores(7개), brief(있으면)
- individuals: 각 팀의 4명에 대해 raw(7개), weighted(7개), indiv_total
누락 항목이 있으면 “missing_fields”에 기록하고, 리포트의 ‘모델 범위/제약’ 섹션에 반영하라.

[STEP 2 — COMPUTE EXPLANATION FEATURES (설득용 파생값)]
(표기값은 로그를 그대로 쓰되) 아래 파생값을 계산해 ‘설명’에만 사용하라:
- team_contrib[k] = weight[k] * team_driver_score[k]  (k=7개)
- team_top_strengths: contrib 상위 2개(단, decision_making은 별도 표기)
- team_main_risk: driver_score 최저 1개 + (필요시 decision_making note)
- individual_contrib[k] = weight[k] * raw[k]
- individual_high/low: raw 기준 상·하위 1~2개 + 가중치 반영(개인 contrib)

[STEP 3 — METHODOLOGY (투명성: 산식+전수탐색) 작성]
다음 내용을 “짧지만 완결되게” 반드시 서술하라:
- 7개 지표 정의(REFERENCE_MATERIALS 기반 요약)
- 팀 점수 산식:
  - 6개 지표는 팀 평균(mean)
  - decision_making은 팀 내 표준편차 기반 ‘균형 점수’(dec_balance)임을 명확히
  - 최종 TeamScore = 100 * Σ(weight * driver_score), 0~100
- 개인 점수 산식:
  - weighted = raw * weight
  - indiv_total = 100 * Σ(weighted)
- 탐색 방법:
  - combinations = C(employees, team_size) (exact) 값을 그대로 제시
  - 전수탐색으로 모든 조합 평가 후 Top-5 선별(Top-k 유지 방식 요약)

[STEP 4 — WEIGHT JUSTIFICATION (도메인 로직으로 설득) 작성]
- 아래 3개 매핑 표를 반드시 작성(REFERENCE_MATERIALS 기반):
  1) Cynefin(단순/난해/복잡/혼란) → 가중치 전략(어떤 지표가 왜 ↑/↓)
  2) Cost of Error(High/Low) → 가중치 전략
  3) Interdependence(Pooled/Reciprocal) → 가중치 전략
- USER_REQUIREMENTS가 있으면: 그 맥락으로 “현재 WEIGHTS가 왜 합리적인지”를 5~7줄로 연결.
- USER_REQUIREMENTS가 없으면:
  - “가정:”으로 현재 WEIGHTS 패턴을 해석해 가능 맥락 2가지를 제시하고,
  - 각 맥락이 의미하는 가중치 방향성과 현재 WEIGHTS가 맞는지/어긋나는지 논리적으로 설명.

[OUTPUT CONTRACT — 반드시 2개 산출물]
출력은 반드시 아래 순서로 2개를 모두 제공한다.

(1) MARKDOWN_REPORT (사람이 읽는 리포트)
- 길이: ‘짧게’가 아니라 ‘리포트답게’(요약+근거+권고 포함). 단, 불필요한 미사여구 금지.
- 고정 섹션/순서(헤더 제목 동일하게 유지):
  1. Executive Summary
     - Top-1 추천 결론(팀명/점수/핵심 근거 3개)
     - Top-5 전체 비교에서 드러나는 패턴 2개(예: 혁신 강세, 의사결정 균형 등)
  2. Inputs & Scope
     - 입력(REFERENCE_MATERIALS/USER_RUN_LOG)과 자동주입 가정
     - 모델 범위/제약(누락 데이터, 측정 범위 등) — “missing_fields” 기반으로 작성
  3. Methodology (Score Transparency)
     - 팀 점수/개인 점수/전수탐색을 짧고 명확하게
  4. Weight Rationale (Domain Logic)
     - 3개 매핑 표 + 현재 WEIGHTS 정당화(또는 가정 시나리오)
  5. Top-5 Overview Dashboard
     - 팀 A~E 카드형 요약(각 카드: TeamScore, 멤버, 강점2, 리스크1)
     - 7개 지표의 팀별 비교표(숫자 + 한줄 해석)
  6. Team Detail — Team A (Rank #1)
     - Team Snapshot: 멤버/팀 점수/WEIGHTS 요약
     - Team Radar 해석(7축)
     - Why this team ranks #1: contrib 기반 근거(가중치×driver 상위 2개 + decision_making 별도)
     - Risks & Warnings: 최저 driver + decision_making 균형 문장
     - Recommended Actions: 3~5개(리스크와 직접 연결)
     - Individuals (4명): 각 개인별 “요약 3줄 + 표 + 레이더(데이터 스펙 참조)” 
  7. Team Detail — Team B (Rank #2)
  8. Team Detail — Team C (Rank #3)
  9. Team Detail — Team D (Rank #4)
  10. Team Detail — Team E (Rank #5)
     - Team B~E는 A와 동일 구조로 작성하되, 불필요한 반복 문장은 줄이고 “차별점(왜 A보다 낮은지)”을 반드시 2줄 포함.
  11. Appendix
     - 용어(7개 지표, decision_making ‘균형 점수’ 정의)
     - 로그 핵심 발췌(짧게)

(2) UI_SPEC_JSON (렌더링용 — 차트/표 데이터)
- JSON은 파싱 가능해야 하며, 키 이름 고정.
- 반드시 포함:
  - meta: employees, team_size, combinations_exact, weights(7개)
  - axes_order 고정:
    ["trust","communication","decision_making","innovative_thinking","conflict_management","role_clarity","psychological_safety"]
  - teams[5]:
    - team_id("A"~"E"), rank, team_score_100, members[4]
    - team_driver_scores_0to1 (7개)
    - team_driver_scores_0to5 (7개: *5 변환)
    - team_contrib (7개: weight*driver)
    - team_radar_spec: {type:"radar", axes_ko, values_0to5, min:0, max:5}
    - individuals[4] 각각:
      - member_id, indiv_total_100
      - raw_0to1(7개), raw_0to5(7개)
      - weighted(7개)
      - individual_contrib(7개: weight*raw)
      - individual_radar_spec: {type:"radar", axes_ko, values_0to5, min:0, max:5}
  - overview_compare:
    - table_teams_x_drivers: 팀×7지표 매트릭스(0~5)
    - notes: 각 팀의 강점2/리스크1 텍스트(로그 기반)

[STYLE RULES]
- 리포트 문장은 “근거 → 해석 → 의미(의사결정)” 순서로 작성한다.
- 팀 간 비교 문장은 반드시 ‘차이’(예: A vs B)로 작성한다. (단순 나열 금지)
- “왜”를 말할 때는 반드시 contrib(가중치×점수) 또는 raw/weighted를 연결해 설명한다.
- 같은 문장 패턴을 반복하지 않는다(문장 변주).
- 과장/감성/홍보 문구 금지. 숫자와 논리로 설득한다.

[FAIL-SAFE]
- USER_RUN_LOG에 특정 팀/개인 항목이 누락되면:
  - 해당 항목은 UI_SPEC_JSON에 null로 넣고,
  - MARKDOWN_REPORT의 ‘Inputs & Scope’에 누락 사실과 영향만 명확히 기록한다."""



user_prompt = """ 팀 빌딩 리포트 만들어줘 """